In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
random_state = 42
import autoreload
%load_ext autoreload
%autoreload 2
import pickle

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,recall_score,precision_score
from treeinterpreter import treeinterpreter as ti
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_csv(r"C:\Users\efann\Desktop\SNAP\final.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3904 entries, 0 to 3903
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FSSLTDED  3904 non-null   float64
 1   SHELDED   3904 non-null   float64
 2   CERTHHSZ  3904 non-null   float64
 3   TANF_IND  3904 non-null   float64
 4   FSNELDER  3904 non-null   float64
 5   FSUNEARN  3904 non-null   float64
 6   FSASSET   3904 non-null   float64
 7   RAWNET    3904 non-null   float64
 8   FSUSIZE   3904 non-null   float64
 9   FSNETINC  3904 non-null   float64
 10  LIQRESOR  3904 non-null   float64
 11  FSSTDDE2  3904 non-null   float64
 12  FSSSI     3904 non-null   float64
 13  TPOV      3904 non-null   float64
 14  HWGT      3904 non-null   float64
 15  WRK_POOR  3904 non-null   float64
 16  FSTOTDE2  3904 non-null   float64
 17  FSDIS     3904 non-null   float64
 18  FSGRINC   3904 non-null   float64
 19  FSERNDE2  3904 non-null   float64
 20  FSEARN    3904 non-null   floa

In [4]:
#Upper case X; all columns except for cat elig- that one is the one we are predicting- the y.
X = df.drop(columns = ['CAT_ELIG'])
y = df['CAT_ELIG']

In [5]:
#baseline, NULL Model; this is a starting point and tells you that your model for predicting y, cat_elig, needs to be higher than .66
y.value_counts(normalize=True)

1.0    0.662398
0.0    0.337602
Name: CAT_ELIG, dtype: float64

In [6]:
#the standard test size is anywhere from .2 to .3 meaning 20 to 30 percent for training, you want most of the data to go to training.  If you get a bad accuracy score, this is a place you can change to see if you can make it better
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=random_state)

In [7]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [8]:
np.savetxt(r'C:\Users\efann\Desktop\SNAP\TrainTest\X_train.csv',X_train,delimiter=',')
np.savetxt(r'C:\Users\efann\Desktop\SNAP\TrainTest\y_train.csv',y_train,delimiter=',')
np.savetxt(r'C:\Users\efann\Desktop\SNAP\TrainTest\X_test.csv',X_test,delimiter=',')
np.savetxt(r'C:\Users\efann\Desktop\SNAP\TrainTest\y_test.csv',y_test,delimiter=',')

In [9]:
#Added this I think i missed it before for bagging
pca = PCA(n_components=10,random_state=42)
pca.fit(X_train)
X_train_pc = pca.transform(X_train)
X_test_pc = pca.transform(X_test)

In [10]:
#the first three are white box models; the can be interpreted; the others are boosters
models = {
    'LogReg': LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier(),
    'Random Forest':RandomForestClassifier(),
    'Gradient Boost':GradientBoostingClassifier(),
    'Ada Boost':AdaBoostClassifier(),
    'SVC':SVC(),
    'Naive Bayes':GaussianNB()}

In [11]:
#adapted from Dan Brown lecture
final = pd.DataFrame(columns = ['cross_val_train','cross_val_test','test_recall','test_precision'])
idx=0
while idx < len(models.keys()):
    for name,model in models.items():
        results = {}
        results['name']=name
        name=model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        results['cross_val_train'] = np.mean(cross_val_score(model,X_train,y_train,cv=4))
        results['cross_val_test'] = np.mean(cross_val_score(model,X_test,y_test,cv=4))
        results['test_recall'] = recall_score(y_test, y_pred_test)
        results['test_precision'] = precision_score(y_test, y_pred_test)
        final = final.append(results,ignore_index=True)
        idx+=1

In [12]:
final.set_index('name')

,cross_val_train,cross_val_test,test_recall,test_precision
name,,,,
LogReg,0.884334,0.862628,0.900262,0.903821
Decision Tree,0.906296,0.912969,0.940945,0.932380
Random Forest,0.943265,0.936007,0.959318,0.958060
Gradient Boost,0.941435,0.933447,0.950131,0.961487
Ada Boost,0.922401,0.920648,0.937008,0.939474
SVC,0.884700,0.854949,0.901575,0.928378
Naive Bayes,0.767204,0.741468,0.833333,0.908441


In [13]:
for name,model in models.items():
    name = model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print(name)
    print(classification_report(y_test, y_pred_test))
    print('\n')

LogisticRegression()
              precision    recall  f1-score   support

         0.0       0.82      0.82      0.82       410
         1.0       0.90      0.90      0.90       762

    accuracy                           0.87      1172
   macro avg       0.86      0.86      0.86      1172
weighted avg       0.87      0.87      0.87      1172



DecisionTreeClassifier()
              precision    recall  f1-score   support

         0.0       0.89      0.90      0.89       410
         1.0       0.95      0.94      0.94       762

    accuracy                           0.93      1172
   macro avg       0.92      0.92      0.92      1172
weighted avg       0.93      0.93      0.93      1172



RandomForestClassifier()
              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92       410
         1.0       0.95      0.96      0.96       762

    accuracy                           0.95      1172
   macro avg       0.94      0.94      0.94      1172

In [14]:
#Testing the random tree
et = ExtraTreeClassifier()
params={'max_depth':[None,3,4],
       'max_features':[None,'auto'],
       'max_leaf_nodes':[5,10]}
et_gs = GridSearchCV(et,param_grid=params)
et_gs.fit(X_train,y_train)
print(et_gs.best_score_)
et_gs.best_params_

0.8425979870221187


{'max_depth': None, 'max_features': None, 'max_leaf_nodes': 10}

In [15]:
#Testing bagging, code didn't work X_train_pc not defined at first but added the missing code
bag = BaggingClassifier()
bag.fit(X_train,y_train)
y_pred_train = bag.predict(X_train)
y_pred_test = bag.predict(X_test)
print(f'cross_val_train = {np.mean(cross_val_score(model,X_train_pc,y_train,cv=4))}')
print(f'cross_val_test = {np.mean(cross_val_score(model,X_test_pc,y_test,cv=4))}')
print(f'test_recall = {recall_score(y_test, y_pred_test)}')
print(f'test_precision = {precision_score(y_test, y_pred_test)}')

cross_val_train = 0.7269399707174231
cross_val_test = 0.7133105802047782
test_recall = 0.9501312335958005
test_precision = 0.9476439790575916


In [16]:
rf = RandomForestClassifier()
params={'max_depth':[None,3,4],
       'max_features':[None,'auto'],
       'n_estimators':[75,100,125]}
rf_gs = GridSearchCV(rf,param_grid=params)
rf_gs.fit(X_train,y_train)
print(rf_gs.best_score_)
rf_gs.best_params_

0.9439962231552727


{'max_depth': None, 'max_features': 'auto', 'n_estimators': 125}

In [17]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [18]:
vote = VotingClassifier([
    ('rf',RandomForestClassifier(bootstrap=False,n_estimators=1000)),
    ('gb',GradientBoostingClassifier(max_depth=10,subsample=0.8)),
    ('bag',BaggingClassifier(n_estimators = 10))
])

In [19]:
vote.fit(X_train,y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=False,
                                                     n_estimators=1000)),
                             ('gb',
                              GradientBoostingClassifier(max_depth=10,
                                                         subsample=0.8)),
                             ('bag', BaggingClassifier())])

In [20]:
filename = 'final_model.sav'
pickle.dump(vote, open(filename, 'wb'))